<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>